In [1]:
!pip install uv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 101.7 MB/s eta 0:00:00


In [2]:
!uv pip install opencv-python torch torchvision -q

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os
import glob
import gc
import torch

МАСКА ИСКЛЮЧЕНИЯ ИЗ ВНИМАНИЯ БАННЕРОВ

In [5]:
def create_text_mask_by_location(image):
    h, w = image.shape[:2]
    # Создаем пустую маску (255 - область интереса, 0 - текст/баннеры)
    text_mask = np.ones((h, w), dtype=np.uint8) * 255

    # Координаты баннеров (исправленные на основе предположений)
    # Первый баннер: [45, 474] (x1, y1) и [334, 524] (x2, y2)
    x1, y1 = min(45, 334), min(474, 524)  # Верхний левый угол
    x2, y2 = max(45, 334), max(474, 524)  # Нижний правый угол
    text_mask[y1:y2, x1:x2] = 0

    # Второй баннер: [879, 2] (x1, y1) и [1072, 50] (x2, y2)
    x1, y1 = min(879, 1072), min(2, 50)  # Верхний левый угол
    x2, y2 = max(879, 1072), max(2, 50)  # Нижний правый угол
    text_mask[y1:y2, x1:x2] = 0


    # Преобразование исходного изображения в HSV для создания маски фона
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Диапазоны для синего фона (настройте под ваш кадр)
    lower_blue = np.array([100, 200, 150])  # Уточненный диапазон для синего
    upper_blue = np.array([130, 255, 255])
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    # Инвертируем маску фона (0 - синий фон, 255 - остальное)
    mask_blue = cv2.bitwise_not(mask_blue)

    # Объединяем маску баннеров и маску фона
    final_mask = cv2.bitwise_and(text_mask, mask_blue)

    return final_mask



МАСКА ДЛЯ "ДОМА"

In [6]:
# Новая функция для создания маски "дома"
def create_house_mask(image, text_mask):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Диапазон для "дома"
    lower_house = np.array([105, 25, 75])
    upper_house = np.array([125, 178, 130])

    mask_house = cv2.inRange(hsv, lower_house, upper_house)

    # Применяем маску баннеров, чтобы исключить текст
    mask_house = cv2.bitwise_and(mask_house, mask_house, mask=text_mask)

    # Морфологические операции для улучшения маски
    kernel = np.ones((5, 5), np.uint8)
    mask_house = cv2.morphologyEx(mask_house, cv2.MORPH_OPEN, kernel)  # Удаление шума
    mask_house = cv2.morphologyEx(mask_house, cv2.MORPH_CLOSE, kernel)  # Заполнение разрывов


    # Поиск контуров "дома"
    contours_house, _ = cv2.findContours(mask_house, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Фильтрация контуров по площади
    filtered_contours_house = [cnt for cnt in contours_house if cv2.contourArea(cnt) > 16000]

    return mask_house, filtered_contours_house

МАСКА ДЛЯ КАМНЕЙ (ЖЁЛТЫХ И КРАСНЫХ)

In [7]:
def find_contours_of_cards(image, text_mask):
    # Применяем маску текста, чтобы исключить баннеры
    image_masked = cv2.bitwise_and(image, image, mask=text_mask)

    # Преобразование в HSV для камней
    hsv = cv2.cvtColor(image_masked, cv2.COLOR_BGR2HSV)

    # Диапазоны для желтых камней
    lower_yellow = np.array([20, 50, 130])
    upper_yellow = np.array([30, 255, 255])
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Диапазоны для красных камней (два диапазона, так как красный цвет в HSV делится на два участка)
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 230, 230])
    lower_red2 = np.array([160, 50, 50])
    upper_red2 = np.array([180, 230, 230])
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)

    # Диапазон для серого (для учета серой части камней)
    lower_gray = np.array([0, 0, 60])
    upper_gray = np.array([50, 100, 180])
    mask_gray = cv2.inRange(hsv, lower_gray, upper_gray)

    # Объединяем маски для желтых и серых частей камней
    mask_yellow_combined = cv2.bitwise_or(mask_yellow, mask_gray)
    # Объединяем маски для красных и серых частей камней
    mask_red_combined = cv2.bitwise_or(mask_red, mask_gray)

    ## Применяем маску текста к каждой цветной маске
    mask_yellow_combined = cv2.bitwise_and(mask_yellow_combined, mask_yellow_combined, mask=text_mask)
    mask_red_combined = cv2.bitwise_and(mask_red_combined, mask_red_combined, mask=text_mask)

    # Морфологические операции для очистки масок
    kernel = np.ones((5, 5), np.uint8)
    mask_yellow_combined = cv2.morphologyEx(mask_yellow_combined, cv2.MORPH_OPEN, kernel)
    mask_yellow_combined = cv2.morphologyEx(mask_yellow_combined, cv2.MORPH_CLOSE, kernel)
    mask_red_combined = cv2.morphologyEx(mask_red_combined, cv2.MORPH_OPEN, kernel)
    mask_red_combined = cv2.morphologyEx(mask_red_combined, cv2.MORPH_CLOSE, kernel)

    # Поиск контуров для желтых камней
    contours_yellow, _ = cv2.findContours(mask_yellow_combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours_yellow = [cnt for cnt in contours_yellow if 400 < cv2.contourArea(cnt) < 1500]

    # Поиск контуров для красных камней
    contours_red, _ = cv2.findContours(mask_red_combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours_red = [cnt for cnt in contours_red if 400 < cv2.contourArea(cnt) < 1500]

    def filter_ellipses(contours, mask_color, mask_gray):
        confirmed_contours = []
        for cnt in contours:
            # Проверяем, достаточно ли точек для аппроксимации эллипса (не менее 5)
            if len(cnt) >= 5:
                x, y, w, h = cv2.boundingRect(cnt)
                # Проверяем, что ROI не пустой
                if w > 0 and h > 0:
                    try:
                        # Аппроксимация эллипса
                        ellipse = cv2.fitEllipse(cnt)
                        (center_x, center_y), (major_axis, minor_axis), angle = ellipse


                        # Проверяем, соответствует ли эллипс размерам контура
                        axis_ratio = min(major_axis, minor_axis) / max(major_axis, minor_axis)

                        if 0.7 < axis_ratio < 1.3:  # Допустимое соотношение осей
                            if abs(major_axis - w) < 15 and abs(minor_axis - h) < 15:
                                # Проверяем наличие цветной части (желтой или красной) в ROI
                                roi_color = mask_color[y:y+h, x:x+w]
                                roi_gray = mask_gray[y:y+h, x:x+w]
                                color_pixels = cv2.countNonZero(roi_color)
                                gray_pixels = cv2.countNonZero(roi_gray)
                                total_pixels = roi_color.size
                                if total_pixels > 0:
                                    color_ratio = color_pixels / total_pixels
                                    gray_ratio = gray_pixels / total_pixels
                                    # Убедимся, что есть цветная часть (желтая/красная) и серая
                                    if (color_ratio > 0.1 and gray_ratio > 0.1) or (color_ratio > 0.1 and gray_ratio > 0.01):
                                        confirmed_contours.append(cnt)
                    except cv2.error:
                        # Пропускаем контуры, для которых не удалось аппроксимировать эллипс
                        continue
        return confirmed_contours

    # Фильтруем контуры с помощью эллипсов
    confirmed_yellow = filter_ellipses(filtered_contours_yellow, mask_yellow, mask_gray)
    confirmed_red = filter_ellipses(filtered_contours_red, mask_red, mask_gray)

    return confirmed_yellow, confirmed_red, mask_yellow_combined, mask_red_combined

СОЗДАНИЕ АННОТАЦИИ

In [8]:
# Функция создания аннотации
def create_yolo_annotations(contours_yellow, contours_red, contours_house, frame_number, image_shape, output_dir):
    img_height, img_width = image_shape[:2]
    annotations_yolo = []
    class_map = {"yellow": 0, "red": 1, "house": 2}  # Добавляем класс для "дома"

    for cnt in contours_yellow:
        x, y, w, h = cv2.boundingRect(cnt)
        center_x = x + w // 2
        center_y = y + h // 2
        norm_center_x = center_x / img_width
        norm_center_y = center_y / img_height
        norm_width = w / img_width
        norm_height = h / img_height
        annotations_yolo.append(f"{class_map['yellow']} {norm_center_x:.6f} {norm_center_y:.6f} {norm_width:.6f} {norm_height:.6f}")

    for cnt in contours_red:
        x, y, w, h = cv2.boundingRect(cnt)
        center_x = x + w // 2
        center_y = y + h // 2
        norm_center_x = center_x / img_width
        norm_center_y = center_y / img_height
        norm_width = w / img_width
        norm_height = h / img_height
        annotations_yolo.append(f"{class_map['red']} {norm_center_x:.6f} {norm_center_y:.6f} {norm_width:.6f} {norm_height:.6f}")


    for cnt in contours_house:
        x, y, w, h = cv2.boundingRect(cnt)
        center_x = x + w // 2
        center_y = y + h // 2
        norm_center_x = center_x / img_width
        norm_center_y = center_y / img_height
        norm_width = w / img_width
        norm_height = h / img_height
        annotations_yolo.append(f"{class_map['house']} {norm_center_x:.6f} {norm_center_y:.6f} {norm_width:.6f} {norm_height:.6f}")

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    txt_output_path = os.path.join(output_dir, f"frame_{frame_number}.txt")
    with open(txt_output_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(annotations_yolo))


In [9]:
# Основная функция создания аннотаций к отобранными кадрам
def create_annotation(selected_frames_dir_path, selected_frames_output_dir, output_annotation_dir):

    number_of_selected_frames = 0
    number_of_created_annotations = 0

    # Получаем список всех .jpg файлов в папке
    frame_paths = glob.glob(os.path.join(selected_frames_dir_path, "*.jpg"))
    # Сортировка по имени файла (опционально, для порядка)
    frame_paths.sort()

      # Загрузка кадров
    selected_frames = [cv2.imread(path) for path in frame_paths]
    if any(ref is None for ref in selected_frames):
        raise ValueError("Ошибка загрузки отобранных кадров")

    # Создание папки для сохранения аннотаций
    os.makedirs(output_annotation_dir, exist_ok=True)

    # Создание папки для сохранения кадров
    os.makedirs(selected_frames_output_dir, exist_ok=True)

    # Создание аннотаций
    for selected_frame, frame_path in zip(selected_frames, frame_paths):

        frame_number = ''

        # Создание масок для кадра
        text_mask = create_text_mask_by_location(selected_frame)
        mask_house, contours_house = create_house_mask(selected_frame, text_mask)

        contours_yellow, contours_red, mask_yellow, mask_red = find_contours_of_cards(selected_frame, text_mask)

        # Проверка на пустую аннотацию
        is_annotation_empty = (len(contours_yellow) == 0 and len(contours_red) == 0 and len(contours_house) == 0)

        if is_annotation_empty:
            print("NO11das")
            continue

        # Номер кадра
        frame_filename = os.path.basename(frame_path)  # Получаем имя файла из пути

        if "angle_0_frame_" in frame_filename:
            frame_number = frame_filename.replace('angle_0_frame_', '').replace('.jpg', '')

        elif "angle_1_frame_" in frame_filename:
            frame_number = frame_filename.replace('angle_1_frame_', '').replace('.jpg', '')


        # Создание и сохранение YOLO аннотации (сохранение включено в create_yolo_annotations)
        create_yolo_annotations(contours_yellow, contours_red, contours_house, frame_number, selected_frame.shape, output_dir=output_annotation_dir)
        number_of_created_annotations += 1

        # Сохранение кадра
        output_path = os.path.join(selected_frames_output_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_path, selected_frame)
        number_of_selected_frames += 1
        print("DAaa")

    print(f"Кадры сохранены в: {selected_frames_output_dir}\n Сохранено кадров: {number_of_selected_frames}")
    print(f"Аннотации сохранены в: {output_annotation_dir}\n Сохранено аннотаций: {number_of_created_annotations}")




In [10]:
# Запуск автоматического создания аннотаций к отобранными кадрам
selected_frames_dir_path = "/content/drive/MyDrive/Sber_Test/Полный_матч_Отобранные_по_углам_кадры_1/"
selected_frames_output_dir = "/content/drive/MyDrive/Sber_Test/1_Полный_матч_images"
output_annotation_dir = "/content/drive/MyDrive/Sber_Test/1_Полный_матч_labels"


create_annotation(selected_frames_dir_path, selected_frames_output_dir, output_annotation_dir)

Выходные данные были обрезаны до нескольких последних строк (5000).
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
DAaa
NO11das
NO11das
NO11das
NO11das
NO11das
NO11das
NO11das
NO11das
NO11das
NO11das
NO11das
NO11das
NO11das
NO11das
NO11d